In [ ]:
!pip install transformers diffusers tensorboardX
!git clone https://github.com/ByeongHyunPak/omni-proj.git

import os
os.chdir('/content/omni-proj/omni_proj')

In [ ]:
import random
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as T

from utils import make_coord, gridy2x_erp2pers, gridy2x_pers2erp

def get_pers_centers(num_cols, phi_centers):
    pers_centers = []
    for i, n_cols in enumerate(num_cols):
        PHI = phi_centers[i]
        for j in np.arange(n_cols):
            theta_interval = 360 / n_cols
            THETA = j * theta_interval + theta_interval / 2
            pers_centers.append((THETA, PHI))
    return pers_centers

def projection(x_inp, projy2x, projx2y, THETA, PHI, FOVy, FOVx, HWy):
    device = x_inp.device
    HWx = x_inp.shape[-2:]

    gridy = make_coord(HWy).to(device)
    gridy2x, masky = projy2x(gridy, HWy, HWx, THETA, PHI, FOVy, FOVx, device)
    gridy2x, masky = gridy2x.view(*HWy, 2), masky.view(1, 1, *HWy)

    y_inp = F.grid_sample(
                x_inp, gridy2x.unsqueeze(0).flip(-1),
                mode='bicubic', padding_mode='reflection',
                align_corners=False).clamp_(x_inp.min(), x_inp.max())
    y_inp = y_inp * masky

    gridx = make_coord(HWx, flatten=False).to(device)
    _, maskx = projx2y(gridx, HWy, HWx, THETA, PHI, FOVy, FOVx, device)
    maskx = maskx.view(1, 1, *HWx)

    return y_inp, masky, maskx

def erp2pers(erp_inp, 
             pers_centers,
             pers_size=(512//8, 512//8)):
    
    # ERP to Perspective
    pers_outs = []
    for THETA, PHI in pers_centers:
        pers_out, _, _ = projection(
            erp_inp, gridy2x_erp2pers, gridy2x_pers2erp, 
            THETA, PHI, FOVy=90, FOVx=360, HWy=pers_size)
        pers_outs.append(pers_out)

    return pers_outs

def pers2erp(pers_inps,
             pers_centers,
             erp_size=(1024//8, 2048//8)):
    
    erp_outs = None
    count = None
    for i, pers_inp in enumerate(pers_inps):
    
        # Perspective to ERP
        THETA, PHI = pers_centers[i]
        erp_out, erp_mask, _ = projection(
            pers_inp, gridy2x_pers2erp, gridy2x_erp2pers,
            THETA, PHI, FOVy=360, FOVx=90, HWy=erp_size)
        
        if erp_outs is None:
            erp_outs = erp_out
        else:
            erp_outs += erp_out
        if count is None:
            count = erp_mask
        else:
            count += erp_mask
    erp_outs = torch.where(count > 0, erp_outs / count, erp_outs)

    return erp_outs, erp_mask

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import os


# 저장할 폴더 경로
save_dir = "/content/pixel_distributions/"

if os.path.exists('/content/pixel_distributions/') is False:
    os.mkdir('/content/pixel_distributions/')


# 초기 노이즈 생성
init_pers_noises = [torch.randn((1, 1, 64, 64)) for i in range(1)]

# 투영된 노이즈 리스트
projected_erp_noises = []
projected_pers_noises = []

# phi, theta 중심값
num_cols = [1, 1, 1, 1]
# phi_centers = [0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0]
phi_centers = [0, 30.0, 60.0, 90.0]
pers_centers = get_pers_centers(num_cols, phi_centers)

# 각 phi, theta 중심에 대해 ERP 및 Pers 노이즈 투영 수행
for THETA, PHI in pers_centers:
    projected_erp_noise, erp_mask = pers2erp(init_pers_noises, [(THETA, PHI)])
    projected_erp_noises.append(([THETA, PHI], projected_erp_noise, erp_mask))
    projected_pers_noise = erp2pers(projected_erp_noise, [(THETA, PHI)])[0]
    projected_pers_noises.append(([THETA, PHI], projected_pers_noise))

# 분포 시각화를 위한 함수 정의
def plot_pixel_distribution(noise_tensor, mask, title, filename):
    # 마스크가 적용된 영역의 픽셀만 가져오기
    masked_data = noise_tensor[mask.bool()].view(-1).cpu().numpy()
    mean, std = masked_data.mean(), masked_data.std()
    
    # 히스토그램과 정규분포 곡선
    plt.figure(figsize=(8, 6))
    sns.histplot(masked_data, kde=True, stat="density", bins=50, color="skyblue", label='Pixel distribution')
    xmin, xmax = plt.xlim()
    x = torch.linspace(xmin, xmax, 100).cpu().numpy()
    p = norm.pdf(x, mean, std)
    plt.plot(x, p, 'r', lw=2, label='Normal distribution')
    plt.title(title)
    plt.xlabel("Pixel Value")
    plt.ylabel("Density")
    plt.legend()
    plt.savefig(os.path.join(save_dir, filename))
    plt.show()
    plt.close()


# 초기 노이즈 분포 저장 (마스크가 없는 경우)
plot_pixel_distribution(init_pers_noises[0], torch.ones_like(init_pers_noises[0], dtype=bool), 'Initial Perspective Noise', 'initial_perspective_noise.png')

# 각 투영된 ERP 및 Perspective 노이즈 분포 저장
for (theta_phi, erp_noise, erp_mask), (_, pers_noise) in zip(projected_erp_noises, projected_pers_noises):
    theta, phi = theta_phi
    erp_filename = f'erp_noise_theta_{theta}_phi_{phi}.png'
    pers_filename = f'pers_noise_theta_{theta}_phi_{phi}.png'
    plot_pixel_distribution(erp_noise, erp_mask, f'ERP Noise at Theta: {theta}, Phi: {phi}', erp_filename)
    plot_pixel_distribution(pers_noise, torch.ones_like(pers_noise, dtype=bool), f'Perspective Noise at Theta: {theta}, Phi: {phi}', pers_filename)


In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import os

# 저장할 폴더 경로
save_dir = "/content/pixel_distributions"
os.makedirs(save_dir, exist_ok=True)

# 초기 노이즈 생성
init_pers_noises = [torch.randn((1, 1, 64, 64)) for i in range(1)]

# 투영된 노이즈 리스트
projected_erp_noises = []
projected_pers_noises = []

# phi, theta 중심값
num_cols = [1, 1, 1, 1]
# phi_centers = [0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0]
phi_centers = [0, 30.0, 60.0, 90.0]
pers_centers = get_pers_centers(num_cols, phi_centers)

# 각 phi, theta 중심에 대해 ERP 및 Pers 노이즈 투영 수행
for THETA, PHI in pers_centers:
    projected_erp_noise, erp_mask = pers2erp(init_pers_noises, [(THETA, PHI)])
    projected_erp_noises.append(([THETA, PHI], projected_erp_noise, erp_mask))
    projected_pers_noise = erp2pers(projected_erp_noise, [(THETA, PHI)])[0]
    projected_pers_noises.append(([THETA, PHI], projected_pers_noise))

# 분포 시각화를 위한 함수 정의
def plot_pixel_distribution(noise_tensor, mask, title, filename):
    # 마스크가 적용된 영역의 픽셀만 가져오기
    masked_data = noise_tensor[mask.bool()].view(-1).cpu().numpy()
    mean, std = masked_data.mean(), masked_data.std()
    
    # 히스토그램과 정규분포 곡선
    plt.figure(figsize=(8, 6))
    sns.histplot(masked_data, kde=True, stat="density", bins=50, color="skyblue", label='Pixel distribution')
    xmin, xmax = plt.xlim()
    x = torch.linspace(xmin, xmax, 100).cpu().numpy()
    p = norm.pdf(x, mean, std)
    plt.plot(x, p, 'r', lw=2, label='Normal distribution')
    plt.title(title)
    plt.xlabel("Pixel Value")
    plt.ylabel("Density")
    plt.legend()
    plt.savefig(os.path.join(save_dir, filename))
    plt.show()
    plt.close()


# 초기 노이즈 분포 저장 (마스크가 없는 경우)
plot_pixel_distribution(init_pers_noises[0], torch.ones_like(init_pers_noises[0], dtype=bool), 'Initial Perspective Noise', f'{save_dir}/initial_perspective_noise.png')

# 각 투영된 ERP 및 Perspective 노이즈 분포 저장
for (theta_phi, erp_noise, erp_mask), (_, pers_noise) in zip(projected_erp_noises, projected_pers_noises):
    theta, phi = theta_phi
    erp_filename = f'{save_dir}/erp_noise_theta_{theta}_phi_{phi}.png'
    pers_filename = f'{save_dir}/pers_noise_theta_{theta}_phi_{phi}.png'
    plot_pixel_distribution(erp_noise, erp_mask, f'ERP Noise at Theta: {theta}, Phi: {phi}', erp_filename)
    plot_pixel_distribution(pers_noise, torch.ones_like(pers_noise, dtype=bool), f'Perspective Noise at Theta: {theta}, Phi: {phi}', pers_filename)


In [ ]:
import random
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as T

from utils import make_coord, gridy2x_erp2pers, gridy2x_pers2erp

def get_pers_centers(num_cols, phi_centers):
    pers_centers = []
    for i, n_cols in enumerate(num_cols):
        PHI = phi_centers[i]
        for j in np.arange(n_cols):
            theta_interval = 360 / n_cols
            THETA = j * theta_interval + theta_interval / 2
            pers_centers.append((THETA, PHI))
    return pers_centers

def projection(x_inp, projy2x, projx2y, THETA, PHI, FOVy, FOVx, HWy):
    device = x_inp.device
    HWx = x_inp.shape[-2:]

    gridy = make_coord(HWy).to(device)
    gridy2x, masky = projy2x(gridy, HWy, HWx, THETA, PHI, FOVy, FOVx, device)
    gridy2x, masky = gridy2x.view(*HWy, 2), masky.view(1, 1, *HWy)

    y_inp = F.grid_sample(
                x_inp, gridy2x.unsqueeze(0).flip(-1),
                mode='bicubic', padding_mode='reflection',
                align_corners=False).clamp_(x_inp.min(), x_inp.max())
    y_inp = y_inp * masky

    gridx = make_coord(HWx, flatten=False).to(device)
    _, maskx = projx2y(gridx, HWy, HWx, THETA, PHI, FOVy, FOVx, device)
    maskx = maskx.view(1, 1, *HWx)

    return y_inp, masky, maskx

def erp2pers(erp_inp, 
             pers_centers,
             pers_size=(512//8, 512//8)):
    
    # ERP to Perspective
    pers_outs = []
    for THETA, PHI in pers_centers:
        pers_out, _, _ = projection(
            erp_inp, gridy2x_erp2pers, gridy2x_pers2erp, 
            THETA, PHI, FOVy=90, FOVx=360, HWy=pers_size)
        pers_outs.append(pers_out)

    return pers_outs

def pers2erp(pers_inps,
             pers_centers,
             erp_size=(1024//8, 2048//8)):
    
    erp_outs = None
    count = None
    for i, pers_inp in enumerate(pers_inps):
    
        # Perspective to ERP
        THETA, PHI = pers_centers[i]
        erp_out, erp_mask, _ = projection(
            pers_inp, gridy2x_pers2erp, gridy2x_erp2pers,
            THETA, PHI, FOVy=360, FOVx=90, HWy=erp_size)
        
        if erp_outs is None:
            erp_outs = erp_out
        else:
            erp_outs += erp_out
        if count is None:
            count = erp_mask
        else:
            count += erp_mask
    erp_outs = torch.where(count > 0, erp_outs / count, erp_outs)

    return erp_outs

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import os


def plot_pixel_distribution(noise_tensor, title, filename):
    # 마스크가 적용된 영역의 픽셀만 가져오기
    masked_data = noise_tensor.view(-1).cpu().numpy()
    mean, std = masked_data.mean(), masked_data.std()
    
    # 히스토그램과 정규분포 곡선
    plt.figure(figsize=(8, 6))
    sns.histplot(masked_data, kde=True, stat="density", bins=50, color="skyblue", label='Pixel distribution')
    xmin, xmax = plt.xlim()
    x = torch.linspace(xmin, xmax, 100).cpu().numpy()
    p = norm.pdf(x, mean, std)
    plt.plot(x, p, 'r', lw=2, label='Normal distribution')
    plt.title(title)
    plt.xlabel("Pixel Value")
    plt.ylabel("Density")
    plt.legend()
    plt.savefig(os.path.join(save_dir, filename))
    plt.show()
    plt.close()

save_dir = "/content/all_pixel_distributions"
os.makedirs(save_dir, exist_ok=True)

num_cols = [3, 5, 3]
phi_centers = [-67.5, 0.0, 67.5]
pers_centers = get_pers_centers(num_cols, phi_centers)

pers_centers
init_pers_latents = [torch.randn((1, 4, 512//8, 512//8)) for i in range(len(pers_centers))]
proj_erp_latent = pers2erp(init_pers_latents, pers_centers, erp_size=(1024//8, 2048//8))
proj_pers_latents = erp2pers(proj_erp_latent, pers_centers, pers_size=(512//8, 512//8))

plot_pixel_distribution(init_pers_latents, 'Initial Perspective Noise', f'{save_dir}/initial_perspective_noise.png')
plot_pixel_distribution(proj_erp_latent, 'Projected ERP Noise', f'{save_dir}/projected_erp_latent.png')
for i, proj_pers_latent in enumerate(proj_pers_latents):
    plot_pixel_distribution(proj_pers_latent, f'Projected Perspective Noise ({proj_pers_latent[i][0]}º,{proj_pers_latent[i][0]}º)')